In [15]:
#!pip install kafka
#!pip install sqlalchemy
#!pip install pandas

In [16]:
#imports
from kafka import KafkaConsumer
from kafka import TopicPartition
from sqlalchemy import create_engine
import pandas as pd
import json

In [17]:
#set parameters
kafka_server = 'localhost:9092'
db_user = 'postgres'
db_password = 'postgres'
db_host = 'localhost'
db_port = '5432'
db_name = 'postgres'

In [18]:
#init consumer
consumer = KafkaConsumer(
    bootstrap_servers=kafka_server,
    auto_offset_reset='earliest',
    value_deserializer=lambda v: json.loads(v.decode('utf-8').replace("'", '"'))
)

In [19]:
#init database
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
try:
    engine.connect()
    print("success")
except SQLAlchemyError as err:
    print("error", err.__cause__)

success


In [20]:
#find current lenght of 'delays' topic
partition_delays = TopicPartition('delays',0)
consumer.assign([partition_delays])

consumer.seek_to_end(partition_delays)
lastOffset = consumer.position(partition_delays)

consumer.seek_to_beginning(partition_delays)

In [21]:
#consume delays
delay_df = pd.DataFrame()

for msg in consumer:
    row=msg.value
    #currently creating a df, TODO: write to db here
    delay_df = pd.concat([delay_df, pd.DataFrame([row])], ignore_index=True)
    if msg.offset == lastOffset - 1:
            break

In [22]:
#find current lenght of 'weather' topic
partition_weather = TopicPartition('weather',0)
consumer.assign([partition_weather])

consumer.seek_to_end(partition_weather)
lastOffset = consumer.position(partition_weather)

consumer.seek_to_beginning(partition_weather)

In [23]:
#consume weather
weather_df = pd.DataFrame()

for msg in consumer:
    row=msg.value
    weather_df = pd.concat([weather_df, pd.DataFrame([row])], ignore_index=True)
    if msg.offset == lastOffset - 1:
            break

In [24]:
consumer.close()

In [25]:
delay_df['start'] = pd.to_datetime(delay_df['start'], format='%d.%m.%Y %H:%M')#
delay_df['end'] = pd.to_datetime(delay_df['end'], format='%d.%m.%Y %H:%M')
delay_df.head()

,id,title,behoben,lines,stations,start,end
0,1777,Fahrtbehinderung\nVerkehrsunfall,True,[O],[Laxenburger Straße/Troststraße],2021-08-17 05:24:00,2021-08-17 06:24:00
1,1777,Fahrtbehinderung\nVerkehrsunfall,True,[O],[Laxenburger Straße/Troststraße],2021-08-17 05:24:00,2021-08-17 06:24:00
2,1777,Fahrtbehinderung\nVerkehrsunfall,True,[O],[Laxenburger Straße/Troststraße],2021-08-17 05:24:00,2021-08-17 06:24:00
3,1775,O Verkehrsunfall,True,[O],"[Arthaberplatz, Bruno-Marek-Allee, Columbuspla...",2021-08-17 05:21:00,2021-08-17 23:59:00
4,1775,O Verkehrsunfall,True,[O],"[Arthaberplatz, Bruno-Marek-Allee, Columbuspla...",2021-08-17 05:21:00,2021-08-17 23:59:00


In [26]:
delay_df.to_sql('delays_manual', engine, if_exists='replace', index=False)

74

In [27]:
weather_df['time'] = pd.to_datetime(weather_df['time'], format='%Y-%m-%d %H:%M:%S')
weather_df.to_sql('weather_manual', engine, if_exists='replace', index=True,index_label='id')

328